In [ ]:
# Utils packages
from math import ceil
from pathlib import Path, PurePath, PurePosixPath

# Data collector packages
from hentai import Utils, Hentai, Option, Sort

# Analysis packages
import pandas as pd
from PIL import Image

## Data collection

We will use the information and media collected on the nhentai.net website. Recording each entry in a 'metadata.csv' file that aggregates information about the hentai, including links to download pages.

In [ ]:
N_SAMPLE = 0 # Number of sample to refetch from hentai.
DATA_PATH = "data" # Directory to drop collected data.
SEARCH_QUERY = "tag:uncensored" # nhentai search query.
HENTAI_IDS = [] # Manual Hentai IDs to download.
COLLECT_MODE = "random" # Sampling mode.

In [ ]:
def get_hentai_info(hentai: Hentai):
    return hentai.dictionary(Option.all())


def get_random_hentai():
    hentai = Utils.get_random_hentai()
    hentai = get_hentai_info(hentai)
    return hentai


def search_hentai(query: str, page: int):
    return Utils.search_by_query(
        query,
        page,
        sort=Sort.PopularMonth
    )

In [ ]:
# Query the nhentai.net API to get N number of random hentai
if COLLECT_MODE == "search":
    collected_hentais_df = pd.DataFrame([
        pd.Series(get_hentai_info(h))
        for hs in [
            search_hentai(SEARCH_QUERY, p)
            for p in range(1, ceil(N_SAMPLE / 25) + 1)
        ]
        for h in hs
    ])
elif COLLECT_MODE == "random":
    collected_hentais_df = pd.DataFrame([
        pd.Series(get_random_hentai())
        for _ in range(N_SAMPLE)
    ])
else:
    collected_hentais_df = pd.DataFrame([
        pd.Series(get_hentai_info(Hentai(h)))
        for h in HENTAI_IDS
    ])
collected_hentais_df

## Read dataset file

In [ ]:
# The metadata NDJSON file that will contain the raw information about the hentai
metadata_path = PurePath(DATA_PATH) / "raw" / "metadata.ndjson"

In [ ]:
hentais_df = pd.read_json(
    metadata_path,
    orient="records",
    lines=True,
    encoding="utf8"
)
hentais_df

## Save collected dataset

In [ ]:
hentais_df = pd.concat([collected_hentais_df, hentais_df], ignore_index=True)
hentais_df = hentais_df.drop_duplicates("id")
hentais_df.to_json(
    metadata_path,
    orient="records",
    lines=True
)

## Download images

In [ ]:
def download_hentai(row: pd.Series):
    # A hentai will be stored based on the gallery ID instead of the medai ID
    hentai_path = Path(DATA_PATH) / "raw" / str(row.id)
    # Since downloading a hentai is an expensive operation, we only
    # download when the hentai ID directory does not exist
    if not hentai_path.is_dir():
        try:
            hentai = Hentai(row.id)
            hentai.download(folder=hentai_path, progressbar=True)
            return True
        except Exception:
            pass
    return False

In [ ]:
downloaded_hentais_df = hentais_df.apply(download_hentai, axis=1)

In [ ]:
print("Number of downloaded hentais : {}".format(downloaded_hentais_df.sum()))

## Remove corrupted images

In [ ]:
def hentai_filename(hentai: pd.Series):
    return [
        str(PurePosixPath(str(hentai["id"])) / PurePath(url).name)
        for url in hentai["image_urls"]
    ]

In [ ]:
hentais_df = hentais_df.assign(
    filename=hentais_df.apply(hentai_filename, axis=1)
)
hentais_df

We will filter every hentais that contain at least one corrupted images, since pages have order dependencies.

In [ ]:
def preprocess_image(filename: str):
    relative_filepath = Path(filename)
    raw_dirpath = Path(DATA_PATH) / "raw" / relative_filepath.parent.stem
    raw_filepath = raw_dirpath / relative_filepath.name
    preproc_dirpath = Path(DATA_PATH) / "preprocessed" / relative_filepath.parent.stem
    preproc_filepath = preproc_dirpath / f"{raw_filepath.stem}.jpg"
    # Preprocess only those images that do not exist in
    # the destination dataset for performance reason
    preproc_dirpath.mkdir(parents=True, exist_ok=True)
    if not preproc_filepath.is_file():
        image = Image.open(raw_filepath)
        # Normalize images to be JPEG compatible
        image = image.convert("RGB")
        image.save(preproc_filepath, "JPEG")
    return str(PurePath(preproc_filepath))


def preprocess_images(hentai: pd.Series):
    return [
        preprocess_image(filename)
        for filename in hentai["filename"]
    ]

In [ ]:
hentais_df.apply(preprocess_images, axis=1)

In [ ]:
hentais_df = hentais_df[hentais_df["filename"].apply(all)]
hentais_df

In [ ]:
# The metadata CSV file that will contain the raw information about the hentai
metadata_path = PurePath(DATA_PATH) / "preprocessed" / "metadata.ndjson"
hentais_df = hentais_df[[
    "id",
    "title",
    "num_favorites",
    "tag",
    "group",
    "parody",
    "character",
    "language",
    "artist",
    "category",
    "num_pages",
    "filename"
]]
hentais_df.to_json(
    metadata_path,
    orient="records",
    lines=True
)
print("Number of preprocessed images :", len(hentais_df))